In [16]:
import pandas as pd
import geonamescache as gmc
import random
import numpy as np

In [17]:
olmkt = pd.read_csv("online_mkt.csv")

In [18]:
# -----------------------------
# Step 1. Load US states & cities from geonamescache
# -----------------------------
gc = gmc.GeonamesCache()
us_states = gc.get_us_states()      # dict: FIPS -> {name, abbr}
cities = gc.get_cities()            # dict: geonameid -> {name, countrycode, admin1code, ...}

# Build state_name → [cities]
state_to_cities = {s["name"]: [] for s in us_states.values()}
for city in cities.values():
    if city["countrycode"] == "US":
        state_fips = city["admin1code"]
        if state_fips in us_states:
            state_name = us_states[state_fips]["name"]
            state_to_cities[state_name].append(city["name"])

# Drop states with no cities (rare)
state_to_cities = {k: v for k, v in state_to_cities.items() if v}

# -----------------------------
# Step 2. Build Brazil state → U.S. state mapping
# -----------------------------
unique_brazil_states = olmkt["state"].unique()
us_state_names = list(state_to_cities.keys())
random.shuffle(us_state_names)  # optional randomization

state_map = {
    br_state: us_state_names[i % len(us_state_names)]
    for i, br_state in enumerate(unique_brazil_states)
}

# -----------------------------
# Step 3. Build Brazil city → U.S. city mapping
# -----------------------------
city_map = {}
for _, row in olmkt.iterrows():
    br_state, br_city = row["state"], row["city"]
    us_state = state_map[br_state]

    if (br_state, br_city) not in city_map:  # assign once
        city_map[(br_state, br_city)] = random.choice(state_to_cities[us_state])

# -----------------------------
# Step 4. Apply mapping
# -----------------------------
olmkt["us_state"] = olmkt["state"].map(state_map)
olmkt["us_city"] = olmkt.apply(lambda r: city_map[(r["state"], r["city"])], axis=1)

# ----------------------------
# Step 5. Check
# ----------------------------
temp = olmkt.loc[:, olmkt.columns.str.contains("state|city")].drop_duplicates()
temp.sort_values('state')


,city,state,us_state,us_city
1952,maceio,alagoas,West Virginia,Martinsburg
5246,macapa,amapa,Colorado,Montrose
732,manaus,amazonas,Missouri,Saint Joseph
4026,feira_de_santana,bahia,Oklahoma,Duncan
366,salvador,bahia,Oklahoma,Shawnee
488,fortaleza,ceara,New York,Selden
244,brasilia,distrito_federal,Mississippi,West Gulfport
5002,serra,espirito_santo,Arizona,Mesa
5856,vila_velha,espirito_santo,Arizona,Goodyear
4392,aparecida_de_goiania,goias,Iowa,Ames


In [19]:
olmkt['percent_app_download_per_pop'] = 100*(olmkt['app_download']/olmkt['population'])

In [21]:
olmkt.to_csv('ol_mkt_us.csv', index = False)